# 1. 读取mnist 数据

In [84]:
import torch
import torchvision
from torch.utils.data import TensorDataset, DataLoader

train = torchvision.datasets.MNIST("../data/", train=True, download=True,
                                   transform=torchvision.transforms.Compose([
                                       torchvision.transforms.ToTensor(),
                                       torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                   ]))
test = torchvision.datasets.MNIST("../data/", train=False, download=True,
                                  transform=torchvision.transforms.Compose([
                                      torchvision.transforms.ToTensor(),
                                      torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                  ]))
print(type(train))
train.data = train.data.reshape((-1, 28 * 28))
test.data = test.data.reshape((-1, 28 * 28))
print(train.data.shape)
print(train.data.shape)




<class 'torchvision.datasets.mnist.MNIST'>
torch.Size([10000])
torch.Size([60000, 784])
torch.Size([60000, 784])


In [86]:
import numpy as np
from torch.optim import Adam, lr_scheduler
from torch.nn import Sequential
import torch.nn as nn
import torch.nn.functional as F

batch_size = 64
max_epoch = 100
dataloader = DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
# test_data = torch.FloatTensor(test_data).cuda()
# test_target = torch.LongTensor(test_target).cuda()

lr = 0.1


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = Sequential(
            nn.Linear(784, 128, bias=True),
            nn.ReLU(),
            nn.Linear(128, 64, bias=True),
            nn.ReLU(),
            nn.Linear(64, 32, bias=True),
            nn.ReLU(),
            nn.Linear(32, 16, bias=True),
            nn.ReLU(),
            nn.Linear(16, 10, bias=True),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.model(x)


model = Model()
model.cuda()
loss_func = nn.NLLLoss()
optimizer = Adam(model.parameters(), lr=0.1)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
train_loss = []
test_loss = []
for epoch in range(max_epoch):
    epoch_loss = []
    epoch_acc = []
    for step, (batch_x, batch_y) in enumerate(dataloader):
        batch_x = batch_x.cuda()
        batch_x = torch.squeeze(batch_x)
        batch_y = batch_y.cuda()
        pred = model(batch_x)
        loss = loss_func(pred, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = torch.eq(pred.argmax(1), batch_y).sum() / len(batch_y)
        epoch_acc.append(acc.cpu().data)
        print(f"step:{step}: loss={loss.cpu().data}, acc={acc}")
    train_loss.append(np.mean(epoch_loss))
    for test_data, test_target in test:
        test_data = torch.squeeze(test_data)
        pred = model(test_data.cuda())
        test_target = test_target
        loss = loss_func(pred, test_target)
        acc = torch.eq(pred.argmax(1), test_target).sum() / len(test_target)
        test_loss.append(loss.cpu().data)
        print(
            f"epoch {epoch}: train_loss: {np.mean(epoch_loss)}, test_loss: {loss.cpu().data}, train_acc={np.mean(epoch_acc) * 100}, test_acc={acc * 100}")








step:0: loss=-0.10071037709712982, acc=0.078125
step:1: loss=-0.12499997764825821, acc=0.125
step:2: loss=-0.09375, acc=0.09375
step:3: loss=-0.125, acc=0.125
step:4: loss=-0.0625, acc=0.0625
step:5: loss=-0.0625, acc=0.0625
step:6: loss=-0.09375, acc=0.09375
step:7: loss=-0.03125, acc=0.03125
step:8: loss=-0.09375, acc=0.09375
step:9: loss=-0.140625, acc=0.140625
step:10: loss=-0.15625, acc=0.15625
step:11: loss=-0.1875, acc=0.1875
step:12: loss=-0.078125, acc=0.078125
step:13: loss=-0.078125, acc=0.078125
step:14: loss=-0.109375, acc=0.109375
step:15: loss=-0.046875, acc=0.046875
step:16: loss=-0.109375, acc=0.109375
step:17: loss=-0.15625, acc=0.15625
step:18: loss=-0.125, acc=0.125
step:19: loss=-0.171875, acc=0.171875
step:20: loss=-0.09375, acc=0.09375
step:21: loss=-0.078125, acc=0.078125
step:22: loss=-0.125, acc=0.125
step:23: loss=-0.140625, acc=0.140625
step:24: loss=-0.09375, acc=0.09375
step:25: loss=-0.109375, acc=0.109375
step:26: loss=-0.09375, acc=0.09375
step:27: loss

KeyboardInterrupt: 